Przetwarzanie Danych
===========

W celu przetwarzania danych wykorzystałem kod z kernela helpers udostępnionego na Kagglu.
Jedyną zmianą jakiej dokonałem było dodanie poniższej linijki w celu przejścia do skali logarytmicznej na melspectrogramie (decybele). Pozwoliło to uzyskać wyniki średnio wyniki lepsze o 3-4%.



In [ ]:
log_spec = librosa.core.power_to_db(spectrogram, ref=np.median

Architektura
=========

Moje rozwiązanie problemu detekcji ptaków na nagraniach, wykorzystuje konwolucyjne sieci neuronowe. 
Konkretniej sieć składającą się z 3 warstw konwolucyjnych oraz dwóch warstw liniowych. 
Warswy konwolucyjne miały kolejno 16, 32 oraz 32 filtry, a także kernele rozmiarów 3x3, 3x3 oraz 2x2. Reszta szczegołów w notatniku FinalModel.

Inne metody
===========

W rozwiązaniu wykorzystałem ensambling. Finalnie użyłem 10 modeli (ta sama architektura, ale inne wagi), które uzyskiwały najlepsze wyniki na zbiorze walidacyjnym. A następnie uśredniłem ich predykcje co pozwoliło uzyskać na kagglu wynik na poziomie ~0.94 (Pojedyncze modele miały wyniki między 0.91 a 0.93 na zbiorze walidacyjnym).

Generator predykcji
===========

In [1]:
import os
os.chdir('.')
from models import *

In [2]:
paths = []
for i in range(1,11):
    paths.append('saved_model/saved{}.pkl'.format(i))

n_models = 10
pretrained = [True] * n_models #, False, False, False, False]
models = [CNN_Wrapper(i+1, pretrained[i], paths[i]) for i in range(n_models)]

1 True saved_model/saved1.pkl
1 True
2 True saved_model/saved2.pkl
2 True
3 True saved_model/saved3.pkl
3 True
4 True saved_model/saved4.pkl
4 True
5 True saved_model/saved5.pkl
5 True
6 True saved_model/saved6.pkl
6 True
7 True saved_model/saved7.pkl
7 True
8 True saved_model/saved8.pkl
8 True
9 True saved_model/saved9.pkl
9 True
10 True saved_model/saved10.pkl
10 True


In [3]:
X_test_d = np.load('test_data.pkl.npy')

vot_clf = VotingClassifier(models)

In [4]:
vot_clf.fit(None, None)

Starting Model 1
This model has already pretrained
Starting Model 2
This model has already pretrained
Starting Model 3
This model has already pretrained
Starting Model 4
This model has already pretrained
Starting Model 5
This model has already pretrained
Starting Model 6
This model has already pretrained
Starting Model 7
This model has already pretrained
Starting Model 8
This model has already pretrained
Starting Model 9
This model has already pretrained
Starting Model 10
This model has already pretrained


In [5]:
labels = vot_clf.predict(X_test_d)

Predicting Model 0
620 304
Predicting Model 1
636 288
Predicting Model 2
595 329
Predicting Model 3
679 245
Predicting Model 4
583 341
Predicting Model 5
585 339
Predicting Model 6
669 255
Predicting Model 7
605 319
Predicting Model 8
601 323
Predicting Model 9
569 355


In [6]:
final_labels = vot_clf.voting(labels)

['sample_id,prediction', '1/1,0.6220773957885328', '1/2,0.19860870552961118', '1/6,0.2753571279029464', '1/7,0.9335150893064763', '1/8,0.6775118378847942', '2/2,0.42717379114091325', '2/3,0.46413578271092015', '2/4,0.7274932993351917', '2/5,0.9881036253202937', '2/9,0.7458687541569637', '3/1,0.5769716423041564', '3/2,0.8143297404425203', '3/6,0.36707550672124645', '3/7,0.05335952444640679', '3/8,0.08576257193533354', '4/3,0.8998787785684154', '4/4,0.8880390908620751', '4/5,0.6512849050946965', '4/6,0.04318090587232315', '4/8,0.1999680409407817', '5/1,0.10540524896559955', '5/3,0.5731928814265825', '5/6,0.5723600385927952', '5/8,0.13655456378821773', '5/9,0.6426209574636462', '6/1,0.14710052044320743', '6/4,0.8874557519097022', '6/5,0.24729837851537684', '6/7,0.5349352726194767', '6/8,0.6032091601562727', '7/0,0.8724194048205343', '7/1,0.6896909164291573', '7/3,0.08776710307617508', '7/7,0.4749824708216739', '7/9,0.5235180270259688', '8/3,0.35537651856999', '8/4,0.5280186530127838', '8/